In [1]:
import pandas as pd

df = pd.read_csv("/Users/ritushetkar/Downloads/hashtag_posts_unnested.csv")

In [2]:
df

,id,text,textLanguage,createTimeISO,isAd,isMuted,diggCount,shareCount,playCount,collectCount,...,authorMeta_musicAuthor,videoMeta_duration,videoMeta_definition,detailedMentions_nickName,hashtags_name,effectStickers_name,effectStickers_stickerStats_useCount,searchHashtag_views,searchHashtag_name,locationMeta_address
0,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18T20:32:13.000Z,False,False,23,0,349,2,...,Lola Young,20,540p,FORMELSKIN,NaN,NaN,NaN,2700000000,hautpflege,NaN
1,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18T20:32:13.000Z,False,False,23,0,349,2,...,Lola Young,20,540p,FORMELSKIN,skincare,NaN,NaN,2700000000,hautpflege,NaN
2,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18T20:32:13.000Z,False,False,23,0,349,2,...,Lola Young,20,540p,FORMELSKIN,formelskin,NaN,NaN,2700000000,hautpflege,NaN
3,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18T20:32:13.000Z,False,False,23,0,349,2,...,Lola Young,20,540p,FORMELSKIN,skin,NaN,NaN,2700000000,hautpflege,NaN
4,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18T20:32:13.000Z,False,False,23,0,349,2,...,Lola Young,20,540p,FORMELSKIN,haut,NaN,NaN,2700000000,hautpflege,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11607,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13T03:59:07.000Z,False,False,58400,331,86000,24800,...,scand7ll,34,540p,NaN,march,NaN,NaN,583200000000,skincare,NaN
11608,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13T03:59:07.000Z,False,False,58400,331,86000,24800,...,scand7ll,34,540p,NaN,march2025,NaN,NaN,583200000000,skincare,NaN
11609,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13T03:59:07.000Z,False,False,58400,331,86000,24800,...,scand7ll,34,540p,NaN,spring,NaN,NaN,583200000000,skincare,NaN
11610,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13T03:59:07.000Z,False,False,58400,331,86000,24800,...,scand7ll,34,540p,NaN,galiciagordon,NaN,NaN,583200000000,skincare,NaN


In [3]:

df['createTimeISO'] = pd.to_datetime(df['createTimeISO'])
# Create new columns
df['date'] = df['createTimeISO'].dt.date
df['month'] = df['createTimeISO'].dt.strftime('%b')   # e.g., 'Feb'
df['year'] = df['createTimeISO'].dt.year
df['week'] = df['createTimeISO'].dt.isocalendar().week


df

,id,text,textLanguage,createTimeISO,isAd,isMuted,diggCount,shareCount,playCount,collectCount,...,hashtags_name,effectStickers_name,effectStickers_stickerStats_useCount,searchHashtag_views,searchHashtag_name,locationMeta_address,date,month,year,week
0,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,NaN,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8
1,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,skincare,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8
2,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,formelskin,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8
3,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,skin,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8
4,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,haut,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11607,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,march,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11
11608,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,march2025,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11
11609,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,spring,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11
11610,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,galiciagordon,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11


In [4]:
import langid
df['hashtags_lang'] = df['hashtags_name'].apply(lambda x: langid.classify(x)[0] if isinstance(x, str) else 'unknown')


In [5]:
df_hashtag_filter = df[df['hashtags_lang'].isin(['en', 'de'])]
df_hashtag_filter.groupby(['hashtags_lang']).count()

,id,text,textLanguage,createTimeISO,isAd,isMuted,diggCount,shareCount,playCount,collectCount,...,hashtags_name,effectStickers_name,effectStickers_stickerStats_useCount,searchHashtag_views,searchHashtag_name,locationMeta_address,date,month,year,week
hashtags_lang,,,,,,,,,,,,,,,,,,,,,
de,1659,1641,1659,1659,1659,1453,1659,1659,1659,1659,...,1659,76,76,1659,1659,0,1659,1659,1659,1659
en,5180,5172,5180,5180,5180,4580,5180,5180,5180,5180,...,5180,211,211,5180,5180,0,5180,5180,5180,5180


In [6]:
df_text_filtered = df[df['textLanguage'].isin(['en', 'de'])]
print(len(df_text_filtered)
)
df_text_filtered

9218


,id,text,textLanguage,createTimeISO,isAd,isMuted,diggCount,shareCount,playCount,collectCount,...,effectStickers_name,effectStickers_stickerStats_useCount,searchHashtag_views,searchHashtag_name,locationMeta_address,date,month,year,week,hashtags_lang
0,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,unknown
1,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,ro
2,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,en
3,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,en
4,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,2,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en
11607,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en
11608,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en
11609,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,24800,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en


In [7]:
print(len(df_hashtag_filter['hashtags_name'].unique())) #
print(len(df['hashtags_name'].unique()))
print(len(df_text_filtered['hashtags_name'].unique()))

2232
3255
2372


#
Hashtags Over Time



In [8]:
df_text_filtered=df_text_filtered.reset_index()

In [9]:
df_text_filtered

,index,id,text,textLanguage,createTimeISO,isAd,isMuted,diggCount,shareCount,playCount,...,effectStickers_name,effectStickers_stickerStats_useCount,searchHashtag_views,searchHashtag_name,locationMeta_address,date,month,year,week,hashtags_lang
0,0,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,unknown
1,1,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,ro
2,2,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,en
3,3,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,en
4,4,7472859679681482006,@FORMELSKIN | spare 40% auf die ersten zwei Mo...,de,2025-02-18 20:32:13+00:00,False,False,23,0,349,...,NaN,NaN,2700000000,hautpflege,NaN,2025-02-18,Feb,2025,8,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9213,11606,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en
9214,11607,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en
9215,11608,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en
9216,11609,7481138692996877574,asmr self care evening #ASMR #selfcare #ballet...,en,2025-03-13 03:59:07+00:00,False,False,58400,331,86000,...,NaN,NaN,583200000000,skincare,NaN,2025-03-13,Mar,2025,11,en


In [10]:
df_text_filtered.columns

Index(['index', 'id', 'text', 'textLanguage', 'createTimeISO', 'isAd',
       'isMuted', 'diggCount', 'shareCount', 'playCount', 'collectCount',
       'commentCount', 'isSlideshow', 'isPinned', 'isSponsored',
       'authorMeta_nickName', 'authorMeta_verified', 'authorMeta_signature',
       'authorMeta_privateAccount', 'authorMeta_ttSeller', 'authorMeta_fans',
       'authorMeta_heart', 'authorMeta_video', 'authorMeta_digg',
       'authorMeta_region', 'authorMeta_musicName', 'authorMeta_musicAuthor',
       'videoMeta_duration', 'videoMeta_definition',
       'detailedMentions_nickName', 'hashtags_name', 'effectStickers_name',
       'effectStickers_stickerStats_useCount', 'searchHashtag_views',
       'searchHashtag_name', 'locationMeta_address', 'date', 'month', 'year',
       'week', 'hashtags_lang'],
      dtype='object')

In [ ]:
df_text_filtered['index', 'id', 'text', 'textLanguage', 'createTimeISO', 'isAd',
       'isMuted', 'diggCount', 'shareCount', 'playCount', 'collectCount',
       'commentCount', 'isSlideshow', 'isPinned', 'isSponsored',
       'authorMeta_nickName', 'authorMeta_verified', 'authorMeta_signature',
       'authorMeta_privateAccount', 'authorMeta_ttSeller', 'authorMeta_fans',
       'authorMeta_heart', 'authorMeta_video', 'authorMeta_digg',
       'authorMeta_region', 'authorMeta_musicName', 'authorMeta_musicAuthor',
       'videoMeta_duration', 'videoMeta_definition',
       'detailedMentions_nickName', 'hashtags_name', 'effectStickers_name',
       'effectStickers_stickerStats_useCount', 'searchHashtag_views',
       'searchHashtag_name', 'locationMeta_address', 'date', 'month', 'year',
       'week', 'hashtags_lang']

In [12]:
# Are there any rows with empty hashtags
empty_hashtags_count = df_text_filtered[df_text_filtered['hashtags_name'].str.strip() == ''].shape[0]
print("Empty hashtag rows:", empty_hashtags_count)


Empty hashtag rows: 0


In [13]:
hashtag_trends = (
    df_text_filtered.groupby(['year', 'week', 'hashtags_name'])
    .size()
    .reset_index(name='count')
    .sort_values(['year', 'week', 'count'], ascending=[False, False, False])
)

hashtag_trends

,year,week,hashtags_name,count
4209,2025,11,skincare,44
3957,2025,11,fyp,27
4218,2025,11,skincareroutine,23
3854,2025,11,beauty,18
4221,2025,11,skincaretips,17
...,...,...,...,...
3,2020,19,hautpflege,1
4,2020,19,kosmetik,1
5,2020,19,lifestyle,1
6,2020,19,vlog,1


In [14]:
top_hashtags_this_week = (
    hashtag_trends[hashtag_trends['week'] == df_text_filtered['week'].max()]
    .groupby('hashtags_name')['count']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)
top_hashtags_this_week


hashtags_name
skincare             4
hautpflege           2
beauty               2
virall               2
asmr                 2
fypage               2
fyp                  2
oilyskin             1
hautwissen           1
highfrequencywand    1
Name: count, dtype: int64

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# --- Assume df_text_filtered is already prepared with columns: 'year', 'month', 'week', 'hashtags_name'

# Widgets
year_dropdown = widgets.Dropdown(
    options=sorted(df_text_filtered['year'].unique()),
    description='Year:',
    value=df_text_filtered['year'].max()
)

month_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df_text_filtered['month'].dropna().unique()),
    description='Month:',
    value="All"
)

week_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df_text_filtered['week'].dropna().unique()),
    description='Week:',
    value="All"
)

# Output area
output = widgets.Output()

def update_graphs(change=None):
    with output:
        output.clear_output()
        
        # Filter by year
        filtered = df_text_filtered[df_text_filtered['year'] == year_dropdown.value]
        
        # Optional month filter
        if month_dropdown.value != "All":
            filtered = filtered[filtered['month'] == month_dropdown.value]
        
        # Optional week filter
        if week_dropdown.value != "All":
            filtered = filtered[filtered['week'] == week_dropdown.value]
        
        # Count hashtags
        top_hashtags = (
        filtered['hashtags_name']
        .value_counts()
        .head(10)
        .reset_index()
        .rename(columns={'index': 'hashtags_name'})
        )
        top_hashtags.columns = ['hashtags_name', 'count']  # Make column names explicit


        if top_hashtags.empty:
            print("No data available for this selection.")
            return

        # --- Matplotlib + Seaborn Bar Chart ---
        plt.figure(figsize=(10, 6))
        sns.barplot(
            data=top_hashtags,
            x='hashtags_name',
            y='count',
            palette='viridis'
        )
        
        # Add count labels on top of bars
        for index, row in top_hashtags.iterrows():
            plt.text(index, row['count'] + 1, str(row['count']), ha='center', va='bottom', fontsize=10)
        
        # Build dynamic title
        title = f"Top Hashtags in {year_dropdown.value}"
        if month_dropdown.value != "All":
            title += f" - {month_dropdown.value}"
        if week_dropdown.value != "All":
            title += f" - Week {week_dropdown.value}"
        
        plt.title(title, fontsize=14)
        plt.xlabel("Hashtag")
        plt.ylabel("Mentions")
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

        # Show the data table
        display(top_hashtags)

# Trigger when widgets are changed
year_dropdown.observe(update_graphs, names='value')
month_dropdown.observe(update_graphs, names='value')
week_dropdown.observe(update_graphs, names='value')

# Display UI
display(widgets.VBox([year_dropdown, month_dropdown, week_dropdown]))
display(output)

# Initial render
update_graphs()


Output()

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension install --py widgetsnbextension --sys-prefix

In [ ]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook"

def plot_hashtag_trends_over_year_1(df, year_selected, top_n=10):
    # Filter by year
    yearly_data = df[df['year'] == year_selected]

    # Find top N hashtags by frequency
    top_hashtags = (
        yearly_data['hashtags_name']
        .value_counts()
        .head(top_n)
        .index
    )

    # Filter to only top hashtags
    top_data = yearly_data[yearly_data['hashtags_name'].isin(top_hashtags)]

    # Group by week and hashtag
    grouped = (
        top_data
        .groupby(['week', 'hashtags_name'])
        .size()
        .reset_index(name='count')
    )

    # Optional: make sure weeks are sorted numerically
    grouped['week'] = grouped['week'].astype(int)

    # Line chart with Plotly
    fig = px.line(
        grouped,
        x='week',
        y='count',
        color='hashtags_name',
        markers=True,
        title=f"Weekly Trend of Top {top_n} Hashtags in {year_selected}",
        labels={'count': 'Mentions', 'week': 'Week Number'}
        )

    fig.update_layout(
        template="plotly_white",  # <<< This ensures white background
        xaxis=dict(tickmode='linear'),
        height=500
        )
    fig.show()


In [ ]:
plot_hashtag_trends_over_year_1(df_text_filtered, year_selected=2023, top_n=5)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def plot_hashtag_trends_over_year(df, year_selected, top_n=10):
    # Filter by year
    yearly_data = df[df['year'] == year_selected]

    # Find top N hashtags by frequency
    top_hashtags = (
        yearly_data['hashtags_name']
        .value_counts()
        .head(top_n)
        .index
    )

    # Filter to only top hashtags
    top_data = yearly_data[yearly_data['hashtags_name'].isin(top_hashtags)]

    # Group by week and hashtag
    grouped = (
        top_data
        .groupby(['week', 'hashtags_name'])
        .size()
        .reset_index(name='count')
    )

    # Optional: make sure weeks are sorted numerically
    grouped['week'] = grouped['week'].astype(int)

    # Set up the color palette for different hashtags
    palette = sns.color_palette("Set2", n_colors=top_n)

    # Create the plot
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=grouped, 
        x='week', 
        y='count', 
        hue='hashtags_name', 

        palette=palette, 
        markers=True
    )

    # Customize the plot
    plt.title(f"Weekly Trend of Top {top_n} Hashtags in {year_selected}", fontsize=16)
    plt.xlabel('Week Number', fontsize=12)
    plt.ylabel('Mentions', fontsize=12)
    plt.legend(title='Hashtags', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
plot_hashtag_trends_over_year(df_text_filtered, year_selected=2024, top_n=5)


In [ ]:
# Ensure matplotlib plots are shown in the notebook
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  # For log transformation

def plot_hashtag_trends_over_year_log(df, year_selected, top_n=10):
    # Filter by year
    yearly_data = df[df['year'] == year_selected]

    # Find top N hashtags by frequency
    top_hashtags = (
        yearly_data['hashtags_name']
        .value_counts()
        .head(top_n)
        .index
    )

    # Filter to only top hashtags
    top_data = yearly_data[yearly_data['hashtags_name'].isin(top_hashtags)]

    # Group by week and hashtag
    grouped = (
        top_data
        .groupby(['week', 'hashtags_name'])
        .size()
        .reset_index(name='count')
    )

    # Optional: make sure weeks are sorted numerically
    grouped['week'] = grouped['week'].astype(int)

    # Apply log transformation to count values (log of 0 is undefined, so we add 1)
    grouped['log_count'] = np.log1p(grouped['count'])
    grouped['smoothed_count'] = grouped['log_count'].rolling(window=5).mean()

    # Plot with smoothed data
    sns.lineplot(data=grouped, x='week', y='smoothed_count', hue='hashtags_name', markers=True)


    # Set up the color palette for different hashtags
    palette = sns.color_palette("Set2", n_colors=top_n)

    # Create the plot
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=grouped, 
        x='week', 
        y='log_count',  # Plot the log of count
        hue='hashtags_name', 
        palette=palette, 
        markers=True
    )
    #sns.lmplot(data=grouped, x='week', y='log_count', hue='hashtags_name', aspect=2, height=6)

    # Customize the plot
    plt.title(f"Weekly Trend of Top {top_n} Hashtags in {year_selected} (Log Scale)", fontsize=16)
    plt.xlabel('Week Number', fontsize=12)
    plt.ylabel('Log(Mentions)', fontsize=12)
    plt.legend(title='Hashtags', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
plot_hashtag_trends_over_year_log(df_text_filtered, year_selected=2024, top_n=5)


In [17]:
df.columns

Index(['id', 'text', 'textLanguage', 'createTimeISO', 'isAd', 'isMuted',
       'diggCount', 'shareCount', 'playCount', 'collectCount', 'commentCount',
       'isSlideshow', 'isPinned', 'isSponsored', 'authorMeta_nickName',
       'authorMeta_verified', 'authorMeta_signature',
       'authorMeta_privateAccount', 'authorMeta_ttSeller', 'authorMeta_fans',
       'authorMeta_heart', 'authorMeta_video', 'authorMeta_digg',
       'authorMeta_region', 'authorMeta_musicName', 'authorMeta_musicAuthor',
       'videoMeta_duration', 'videoMeta_definition',
       'detailedMentions_nickName', 'hashtags_name', 'effectStickers_name',
       'effectStickers_stickerStats_useCount', 'searchHashtag_views',
       'searchHashtag_name', 'locationMeta_address', 'date', 'month', 'year',
       'week', 'hashtags_lang'],
      dtype='object')

In [ ]:
#'diggCount', 'shareCount', 'playCount', 'collectCount', 'commentCount'

df_engagement = df_text_filtered.groupby(['year', 'week', 'hashtags_name'])[
    ['diggCount', 'shareCount', 'playCount', 'collectCount', 'commentCount']
].sum().reset_index()

df_engagement

,year,week,hashtags_name,diggCount,shareCount,playCount,collectCount,commentCount
0,2020,19,beauty,59100,247,637300,4320,493
1,2020,19,fy,59100,247,637300,4320,493
2,2020,19,gegenpickel,59100,247,637300,4320,493
3,2020,19,hautpflege,59100,247,637300,4320,493
4,2020,19,kosmetik,59100,247,637300,4320,493
...,...,...,...,...,...,...,...,...
4293,2025,11,yesstyle_ar,58,0,674,7,2
4294,2025,11,youthfulglow,17,1,300,0,3
4295,2025,11,zoyakanwal4004,1023,7,13800,82,48
4296,2025,11,اكسبلور,11,18,329,0,10


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# --- Add a new column: total engagement per post
df_text_filtered['engagement_score'] = ((
    df_text_filtered['diggCount'] + 
    df_text_filtered['shareCount'] + 
    df_text_filtered['playCount'] + 
    df_text_filtered['collectCount'] + 
    df_text_filtered['commentCount']
)/5
)

# --- Widgets
year_dropdown = widgets.Dropdown(
    options=sorted(df_text_filtered['year'].unique()),
    description='Year:',
    value=df_text_filtered['year'].max()
)

month_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df_text_filtered['month'].dropna().unique()),
    description='Month:',
    value="All"
)

week_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df_text_filtered['week'].dropna().unique()),
    description='Week:',
    value="All"
)

output = widgets.Output()

# --- Update function
def update_graphs(change=None):
    with output:
        output.clear_output()
        
        # Filter by dropdowns
        filtered = df_text_filtered[df_text_filtered['year'] == year_dropdown.value]
        if month_dropdown.value != "All":
            filtered = filtered[filtered['month'] == month_dropdown.value]
        if week_dropdown.value != "All":
            filtered = filtered[filtered['week'] == week_dropdown.value]

        # Group and sum engagement per hashtag
        top_engagement = (
            filtered.groupby('hashtags_name')['engagement_score']
            .sum()
            .sort_values(ascending=False)
            .head(10)
            .reset_index()
        )

        if top_engagement.empty:
            print("No data available for this selection.")
            return

        # --- Plot
        plt.figure(figsize=(10, 6))
        sns.barplot(
            data=top_engagement,
            x='hashtags_name',
            y='engagement_score',
            palette='magma'
        )

        # Add value labels
        for index, row in top_engagement.iterrows():
            plt.text(index, row['engagement_score'] + 0.05 * row['engagement_score'], 
                     int(row['engagement_score']), ha='center', fontsize=9)

        # Title
        title = f"Top Hashtags by Engagement in {year_dropdown.value}"
        if month_dropdown.value != "All":
            title += f" - {month_dropdown.value}"
        if week_dropdown.value != "All":
            title += f" - Week {week_dropdown.value}"

        plt.title(title, fontsize=14)
        plt.xlabel("Hashtag")
        plt.ylabel("Total Engagement")
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.show()

        # Show data table
        display(top_engagement)

# Attach interactivity
year_dropdown.observe(update_graphs, names='value')
month_dropdown.observe(update_graphs, names='value')
week_dropdown.observe(update_graphs, names='value')

# Display UI
display(widgets.VBox([year_dropdown, month_dropdown, week_dropdown]))
display(output)

# Initial call
update_graphs()


Output()